In [0]:
import json
import random

In [0]:
class Sample:
  help_text = '\n'.join([
    'Type A, B, and C in order of preference.',
    'For example, if you think B is the best and A is the worst,',
    'type "B A C".',
  ])
  
  def __init__(self, prompt, truth, random, sentiment, rank=None):
    self.prompt = prompt
    self.truth = truth
    self.random = random
    self.sentiment = sentiment
    self.rank = rank
    
  @staticmethod
  def load_file(path):
    '''
    Create a list of Samples from a json file.
    The json should be either a single dict or a list of dicts.
    Each dict must contain the keys "prompt", "truth", "random", and "sentiment".
    '''
    with open(path) as f:
      j = json.load(f)
    if isinstance(j, dict):
      j = [j]
    return [Sample.load_dict(d) for d in j]
  
  @staticmethod
  def load_dict(d):
    return Sample(d['prompt'], d['truth'], d['random'], d['sentiment'],
                  d.get('rank'))
  
  def evaluate(self):
    def print_header(text=None, c='='):
      if not text:
        print(c * 62)
        return
      left = max(30 - len(text) // 2, 10)
      right = max(30 - (len(text) + 1) // 2, 10)
      print(c * left, text, c * right)
  
    print_header('Prompt')
    print(self.prompt)
    completions = [self.truth, self.random, self.sentiment]
    indices = list('ABC')
    random.shuffle(indices)
    mapping = {i: ci for i, ci in zip(indices, range(3))}
    for i in 'ABC':
      print_header(f'Completion {i}', c='-')
      print(completions[mapping[i]])
    
    while True:
      print_header(c='-')
      res = input(f'Rank samples ([h]elp, [q]uit) >>> ')
      if res.startswith('h'):
        print(self.help_text)
        continue
      if res.startswith('q'):
        return None
      res = res.split()
      if len(res) != 3 or set(res) != {'A', 'B', 'C'}:
        print('Invalid rank, try again.')
      else:
        break
    
    rank = []
    for i in res:
      ci = mapping[i]
      rank.append(['truth', 'random', 'sentiment'][ci])
    self.rank = rank
  
  def __str__(self):
    j = {
        'prompt': self.prompt,
        'truth': self.truth,
        'random': self.random,
        'sentiment': self.sentiment,
    }
    if self.rank:
      j['rank'] = self.rank
    return json.dumps(j, indent=4)

In [4]:
s = Sample('the prompt', 'the truth', 'the random', 'the sentiment')
s.evaluate()

print(Sample.load_dict(json.loads(str(s))))

=========================== Prompt ===========================
the prompt
------------------------ Completion A ------------------------
the truth
------------------------ Completion B ------------------------
the random
------------------------ Completion C ------------------------
the sentiment
--------------------------------------------------------------
Rank samples ([h]elp, [q]uit) >>> h
Type A, B, and C in order of preference.
For example, if you think B is the best and A is the worst,
type "B A C".
--------------------------------------------------------------
Rank samples ([h]elp, [q]uit) >>> B C A
{
    "prompt": "the prompt",
    "truth": "the truth",
    "random": "the random",
    "sentiment": "the sentiment",
    "rank": [
        "random",
        "sentiment",
        "truth"
    ]
}
